In [61]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
!pip install timm
!pip install scikit-learn
!pip install tqdm
!pip install torchmetrics


In [63]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve


In [64]:
class ForgeryDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        row = self.labels.iloc[idx]
        img_path = os.path.join(self.root_dir, row['image_path'])
        label = 1 if row['label']=='fake' else 0
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [65]:
IMG_SIZE = 224
BATCH_SIZE = 16

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# Replace with your paths
TRAIN_CSV = "/content/Forgery_Dataset/train_labels.csv"
ROOT_DIR = "/content/Forgery_Dataset"

dataset = ForgeryDataset(TRAIN_CSV, ROOT_DIR, transform=transform)

# Simple 80-20 split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [66]:
class FrequencyHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(112*112, 512),  # <-- CHANGE THIS
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
        )

    def forward(self, x):
        gray = torch.mean(x, dim=1)  # b x H x W
        fft = torch.fft.fft2(gray)
        fft_shift = torch.fft.fftshift(fft)
        mag = torch.log1p(torch.abs(fft_shift))
        b, H, W = mag.shape
        mag_crop = mag[:, :H//2, :W//2]  # 112x112
        mag_flat = mag_crop.reshape(b, -1)  # flatten
        out = self.fc(mag_flat)
        return out


In [67]:
import timm

class HybridDetector(nn.Module):
    def __init__(self, backbone_name='tf_efficientnet_b3_ns', pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0, global_pool='')
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.spatial_head = nn.Sequential(
            nn.Linear(1536, 128),
            nn.ReLU(),
        )
        self.freq_head = FrequencyHead()
        self.fc = nn.Sequential(
            nn.Linear(1536 + 128 + 128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        last = self.backbone(x)
        pooled = self.pool(last).view(last.size(0), -1)
        spat = self.spatial_head(pooled)
        freq = self.freq_head(x)
        concat = torch.cat([pooled, spat, freq], dim=1)
        out = self.fc(concat).squeeze(1)
        return out


In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HybridDetector().to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.6)


/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


In [69]:
def epoch_metrics(scores, labels):
    scores = np.array(scores)
    labels = np.array(labels)
    # Accuracy
    preds = (scores > 0.5).astype(int)
    acc = (preds == labels).mean()
    # AUC
    auc = roc_auc_score(labels, scores)
    # EER
    fpr, tpr, thresholds = roc_curve(labels, scores)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.absolute((fnr - fpr)))]
    eer = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    return {'acc': acc, 'auc': auc, 'eer': eer}


In [70]:
EPOCHS = 8
best_auc = 0.0
history = {'train_loss': [], 'val_loss': [], 'val_acc': [], 'val_auc': [], 'val_eer': []}

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}/{EPOCHS}"):
        imgs = imgs.to(device)
        labels = labels.float().to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)

    scheduler.step()
    train_loss = running_loss / len(train_dataset)

    # validation
    model.eval()
    val_loss = 0.0
    all_scores = []
    all_labels = []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels.float())
            val_loss += loss.item() * imgs.size(0)
            all_scores.extend(outputs.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

    val_loss = val_loss / len(val_dataset)
    metrics = epoch_metrics(all_scores, all_labels)
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(metrics['acc'])
    history['val_auc'].append(metrics['auc'])
    history['val_eer'].append(metrics['eer'])

    print(f"Epoch [{epoch+1}/{EPOCHS}] TrainLoss: {train_loss:.4f} ValLoss: {val_loss:.4f} ValAcc: {metrics['acc']:.4f} ValAUC: {metrics['auc']:.4f} EER: {metrics['eer']:.4f}")

    # save best model
    if metrics['auc'] > best_auc:
        best_auc = metrics['auc']
        torch.save(model.state_dict(), "/content/drive/MyDrive/hybrid_detector_best.pth")
        print("Saved best model to Drive (new best AUC).")


Train Epoch 1/8: 100%|██████████| 1000/1000 [03:40<00:00,  4.53it/s]


Epoch [1/8] TrainLoss: 0.1583 ValLoss: 0.0454 ValAcc: 0.9858 ValAUC: 0.9985 EER: 0.0144
Saved best model to Drive (new best AUC).


Train Epoch 2/8: 100%|██████████| 1000/1000 [03:34<00:00,  4.66it/s]


Epoch [2/8] TrainLoss: 0.0335 ValLoss: 0.0284 ValAcc: 0.9898 ValAUC: 0.9996 EER: 0.0095
Saved best model to Drive (new best AUC).


Train Epoch 3/8: 100%|██████████| 1000/1000 [03:35<00:00,  4.65it/s]


...

Epoch [6/8] TrainLoss: 0.0260 ValLoss: 0.0215 ValAcc: 0.9928 ValAUC: 0.9998 EER: 0.0060


Train Epoch 7/8: 100%|██████████| 1000/1000 [03:34<00:00,  4.66it/s]


Epoch [7/8] TrainLoss: 0.0170 ValLoss: 0.0123 ValAcc: 0.9952 ValAUC: 0.9999 EER: 0.0045
Saved best model to Drive (new best AUC).


Train Epoch 8/8: 100%|██████████| 1000/1000 [03:35<00:00,  4.65it/s]


Epoch [8/8] TrainLoss: 0.0145 ValLoss: 0.0139 ValAcc: 0.9965 ValAUC: 0.9999 EER: 0.0040
